## Simulation with using influx

## 2500m inclined well

In [ ]:
import openlab
import matplotlib.pyplot as plt
import numpy as np
import time
import pandas as pd
import time
import random
import pprint as pr
from IPython.display import clear_output
import datetime
import os
import subprocess

In [ ]:
# Warnings are provided to warn the developer of situations that aren’t necessarily exceptions. 
#Usually, a warning occurs when there is some obsolete of certain programming elements.
#Python program terminates immediately if an error occurs. Conversely, a warning is not critical.

import warnings
warnings.filterwarnings('ignore')

### Logging in and creating session

In [ ]:
username=""
apikey=""
licenseguid=""

In [ ]:
session = openlab.http_client(username=username, apikey=apikey,licenseguid=licenseguid) #this intializes an http client that can be used for an entire openlab session

### Simulation information

In [ ]:
# each simulation lasts 2 minutes, and there are to be 98 simulations with varying setpoints.
single_simulation_lenght = 10 #min
total_simulation_length = 10/60 #hrs

step_per_sim = single_simulation_lenght*60
nr_sim = int(total_simulation_length*60/single_simulation_lenght)
total_steps = nr_sim*step_per_sim

print(f"There will be {nr_sim} simulations, lasting for {single_simulation_lenght} minutes, equalling to {total_steps} timeSteps.")

### Manual Influx Model

In [ ]:
sim_name = "influx_validation"
config_name = "InclinedWell"
initial_bit_depth = 2500
influx_mode = openlab.default_manual_influx

#### Starting simulation of manual influx model

In [ ]:
manual_influx_loss_mass_rate = random.randint(0, 500) / 60
manual_influx_loss_total_mass = random.randint(0, 2000)

sim = session.create_simulation(
    config_name, sim_name, 
    initial_bit_depth, 
    influx_mode=influx_mode, 
    ComplexReservoirKickOffTime=300, 
    Id="ManualInflux", 
    ManualInfluxLossMD=2505, 
    ManualInfluxLossMassRate=manual_influx_loss_mass_rate, #Max = 4999 kg/min
    ManualInfluxLossTotalMass=manual_influx_loss_total_mass, #Max = 5000 kg
    ManualReservoirMode=True, 
    UseReservoirModel=True
)


### Unit Converter

In [ ]:
FLOW_UNIT_CONV_FACTOR= 1.66666667 * 0.00001#float("10e-5") # l/min --> m^3/s
PRESSURE_CONV_FACTOR= 100000.0 # float("10e5") # bar-->pascal
WEIGHT_CONV_FACTOR = 0.001 # float ("10e3") # kg to tons

In [ ]:
print(openlab.setpoints.all_setpoints())

In [ ]:
# variable setpoints
# random lists
SurfaceRPM = [random.randint(0, 220) for _ in range(nr_sim)]
SurfaceRPM = [rpm/60 for rpm in SurfaceRPM]

DesiredWOB = [random.randint(0, 40) for _ in range(nr_sim)]
DesiredWOB = [wob*1000 for wob in DesiredWOB]

FlowRateIn = [random.randint(0, 3000) for _ in range(nr_sim)]
FlowRateIn = [flow_rate/60000 for flow_rate in FlowRateIn]

### Results which are relevant

In [ ]:
print(openlab.results.all_results())

In [ ]:
# dataframe and tags stuff
tags = ['SPP', 'DownholeECD', 'FlowRateOut', 'HookLoad', 'SurfaceTorque', 'BitDepth', 'ChokeOpening', 'DownholePressure', 'ChokePressure', 'FluidTemperatureOut', 'WOB', 'InstantaneousROP', 'BopChokeOpening', 'FlowRateIn', 'TopOfStringVelocity', 'SurfaceRPM', 'ChokePumpFlowRateIn', 'DrillstringTemperature', 'TotalInfluxMass', 'CalculatedPressureBottomHole', 'CuttingsMassFractionTransient', 'GasVolumeFraction', 'DrillstringBucklingLimit', 'FluidTemperatureIn', 'AnnulusECD', 'DrillstringTorqueLimit', 'AnnulusTemperature', 'DrillstringTension', 'AnnulusFluidVelocity', 'DrillstringFluidVelocity', 'AnnulusDensity', 'DrillstringTorque', 'TotalMudLossMass', 'TopOfStringPosition', 'ActivePitVolume', 'ActivePitDensity', 'ActivePitTemperature', 'MainPitVolume', 'MainPitDensity', 'MainPitTemperature', 'GasFlowRateOut', 'DrillstringDensity']
ID = ["sim_ID", "timeStep"]
columns = tags + ID


df = pd.DataFrame(columns=columns)

### Running through the simulations

In [ ]:
startTime = 1
endTime = single_simulation_lenght*60

timeStep = 1
current_simulation = 0

sim.stop()

while current_simulation < nr_sim:
    
    print(f"Simulation {current_simulation + 1}/{nr_sim}")
    
    sim_ID = current_simulation
    
    manual_influx_loss_mass_rate = random.randint(0, 500) / 60
    manual_influx_loss_total_mass = random.randint(0, 2000)

    sim = session.create_simulation(
        config_name, sim_name, 
        initial_bit_depth, 
        influx_mode=influx_mode, 
        ComplexReservoirKickOffTime=300, 
        Id="ManualInflux", 
        ManualInfluxLossMD=2505, 
        ManualInfluxLossMassRate=manual_influx_loss_mass_rate, #Max = 4999 kg/min
        ManualInfluxLossTotalMass=manual_influx_loss_total_mass, #Max = 5000 kg
        ManualReservoirMode=True, 
        UseReservoirModel=True
    )
    
    sim.setpoints.SurfaceRPM = SurfaceRPM[0+current_simulation]
    sim.setpoints.DesiredWOB = DesiredWOB[0+current_simulation]
    sim.setpoints.FlowRateIn = FlowRateIn[0+current_simulation]
    sim.setpoints.TopOfStringVelocity = 0.02 #m/s
    sim.setpoints.DesiredROP = 0.02
    values = {}
    while timeStep <= endTime:

        sim.step(timeStep) # stepping openlab
        sim.get_results(timeStep, tags) # fetching results

        for feature in tags:
            # Skip "Connection" feature
            if feature == "Connection":
                continue
            # Create the variable name for the current feature
            var_name = f"{feature}"
            # Extract the value of the feature at this time step
            value_dict = eval(f"sim.results.{var_name}")
            value = list(value_dict.values())[-1]  # Extract the value as a number
            # Add the value to the dictionary
            values[feature] = value

        # Update other variables based on the current time step and simulation ID, and add them to the dictionary
        values['timeStep'] = timeStep
        values['sim_ID'] = sim_ID

        # Append the dictionary to the dataframe
        df = df.append(values, ignore_index=True)
        print(df.tail(1))
        clear_output(wait=True)
        
        # Increment the time step
        timeStep += 1

    # Print the entire dataframe after the inner while loop has completed
    print(df)
    
    sim.stop()
    timeStep = 1
    current_simulation += 1

print("All simulations completed.")

### A plot to see that the DownholePressure graph is as it should be

In [ ]:
# Extract the downhole pressure and timeStep from the df DataFrame
downhole_pressure = df['DownholePressure']
time_step = df['timeStep']

# Create a line plot
plt.plot(time_step, downhole_pressure)

# Add labels and title
plt.xlabel('Time (s)')
plt.ylabel('Downhole Pressure (psi)')
plt.title('Downhole Pressure vs Time')

# Show the plot
plt.show()


### Saving the df to a csv file, change to your folder directory

In [ ]:
# specify the full path to the folder where the file is saved
folder_path = r"C:

# specify the simulation identifier used to save the file
simulation_id = "influx_validation"

# specify the current date and time as a string
current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# create the folder if it does not exist
os.makedirs(folder_path, exist_ok=True)

# construct the full path to the CSV file to write
file_name = f"results_{simulation_id}_{current_time}.csv"
file_path = os.path.join(folder_path, file_name)

# write the DataFrame to a CSV file
df.to_csv(file_path, index=False)

# read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# open the CSV file using the default program associated with CSV files
if os.name == 'nt':  # for Windows
    os.startfile(file_path)
else:  # for Mac and Linux
    subprocess.call(('open', file_path))